In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np
from missingpy import MissForest
import matplotlib.pylab as plt
import seaborn as sns

In [2]:
# a dictionary to put all of the accuracies
accuracies = {}

In [3]:
# loading data with 10% missing
gse111631_10 = pd.read_csv('data\out\gse111631_10.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(gse111631_10.isna().sum().sum()/(gse111631_10.shape[0]*gse111631_10.shape[1])*100, '% missing values')
print(gse111631_10.shape)
gse111631_10.head()

10.079710144927537 % missing values
(138, 100)


,cg26312252,cg16010788,cg27327268,cg14188232,cg26180255,cg04493788,cg22832061,cg07153419,cg05580991,cg04758270,...,cg16365799,cg10169393,cg01701742,cg26502215,cg02062799,cg26146988,cg27649089,cg07507559,cg25598685,cg24181266
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R01C01,0.946,0.888,0.952,0.066,NaN,0.072,0.018,0.694,0.761,0.899,...,0.019,0.241,0.953,0.919,0.930,0.919,0.018,NaN,0.881,0.040
201172560029_R03C01,0.950,0.870,0.966,0.054,0.893,0.069,0.021,0.793,0.789,0.889,...,0.019,0.184,0.959,0.930,0.936,NaN,0.022,0.930,0.893,0.297
201172560029_R04C01,0.953,0.867,0.963,0.126,0.920,0.047,0.018,NaN,0.782,0.917,...,0.017,0.287,0.961,NaN,0.953,0.925,NaN,0.927,0.847,0.066
201172560029_R05C01,NaN,0.861,0.969,0.049,0.927,0.059,NaN,0.824,0.877,0.953,...,0.022,0.334,0.967,0.904,0.948,0.945,0.019,0.921,NaN,0.029
201172560029_R06C01,0.949,0.846,0.966,NaN,0.941,0.066,0.016,0.845,0.846,0.938,...,0.028,0.406,0.955,NaN,0.949,0.941,0.019,0.942,0.895,NaN


In [4]:
# loading original
original_gse111631 = pd.read_csv('data\out\gse111631_original.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(original_gse111631.shape)
original_gse111631.head()

(138, 100)


,cg26312252,cg16010788,cg27327268,cg14188232,cg26180255,cg04493788,cg22832061,cg07153419,cg05580991,cg04758270,...,cg16365799,cg10169393,cg01701742,cg26502215,cg02062799,cg26146988,cg27649089,cg07507559,cg25598685,cg24181266
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R01C01,0.946,0.888,0.952,0.066,0.912,0.072,0.018,0.694,0.761,0.899,...,0.019,0.241,0.953,0.919,0.930,0.919,0.018,0.871,0.881,0.040
201172560029_R03C01,0.950,0.870,0.966,0.054,0.893,0.069,0.021,0.793,0.789,0.889,...,0.019,0.184,0.959,0.930,0.936,0.909,0.022,0.930,0.893,0.297
201172560029_R04C01,0.953,0.867,0.963,0.126,0.920,0.047,0.018,0.871,0.782,0.917,...,0.017,0.287,0.961,0.880,0.953,0.925,0.019,0.927,0.847,0.066
201172560029_R05C01,0.943,0.861,0.969,0.049,0.927,0.059,0.014,0.824,0.877,0.953,...,0.022,0.334,0.967,0.904,0.948,0.945,0.019,0.921,0.791,0.029
201172560029_R06C01,0.949,0.846,0.966,0.067,0.941,0.066,0.016,0.845,0.846,0.938,...,0.028,0.406,0.955,0.866,0.949,0.941,0.019,0.942,0.895,0.102


In [5]:
def get_accuracies(imputed, missing=gse111631_10, original=original_gse111631):
    # sum of squared errors, mean of average absolute errors, root mean squared error
    ret = {}
    ret['sse'] = ((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().sum()
    ret['mae'] = (imputed.mask(~missing.isna()) - original.mask(~missing.isna())).abs().sum().mean()
    ret['rmse'] = (((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().mean())**(1/2)
    return ret

## HOLOclean

In [9]:
holoclean_betas = pd.read_csv('data\out\holoclean_output_10.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample').drop(columns=['_tid_'])
holoclean_betas.index = holoclean_betas.index.str.upper()
print(holoclean_betas.shape)
holoclean_betas.head()

(138, 100)


,cg26312252,cg16010788,cg27327268,cg14188232,cg26180255,cg04493788,cg22832061,cg07153419,cg05580991,cg04758270,...,cg16365799,cg10169393,cg01701742,cg26502215,cg02062799,cg26146988,cg27649089,cg07507559,cg25598685,cg24181266
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R01C01,0.946,0.888,0.952,0.066,0.923,0.072,0.018,0.694,0.761,0.899,...,0.019,0.241,0.953,0.919,0.930,0.919,0.018,0.906,0.881,0.040
201172560029_R03C01,0.950,0.870,0.966,0.054,0.893,0.069,0.021,0.793,0.789,0.889,...,0.019,0.184,0.959,0.930,0.936,0.930,0.022,0.930,0.893,0.297
201172560029_R04C01,0.953,0.867,0.963,0.126,0.920,0.047,0.018,0.777,0.782,0.917,...,0.017,0.287,0.961,0.918,0.953,0.925,0.019,0.927,0.847,0.066
201172560029_R05C01,0.941,0.861,0.969,0.049,0.927,0.059,0.017,0.824,0.877,0.953,...,0.022,0.334,0.967,0.904,0.948,0.945,0.019,0.921,0.859,0.029
201172560029_R06C01,0.949,0.846,0.966,0.052,0.941,0.066,0.016,0.845,0.846,0.938,...,0.028,0.406,0.955,0.930,0.949,0.941,0.019,0.942,0.895,0.034


In [10]:
holoclean_accuracies = get_accuracies(holoclean_betas, gse111631_10, original_gse111631)
holoclean_accuracies

{'sse': 5.349302999999999,
 'mae': 0.43760999999999994,
 'rmse': 0.2312856026647573}

In [11]:
accuracies['holoclean_aimnet'] = holoclean_accuracies